# Install gdown Python package.

In [0]:
!pip install -U --no-cache-dir gdown

# Install Tensorflow-addons.
* InstanceNormalization

In [0]:
!pip install tensorflow-addons

# Install Tensorflow-datasets.
* celeb_a dataset

In [0]:
!pip install tensorflow-datasets

# Import TensorFlow 2.x.

In [0]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
from tensorflow.keras import backend as K

import numpy as np
np.random.seed(7)

import matplotlib.pyplot as plot

print(tf.__version__)

# Configuration parameters.

In [0]:
auto_tune = tf.data.experimental.AUTOTUNE

In [0]:
generator_dimension = 64
generator_residual_blocks = 6

In [0]:
buffer_size = 512
batch_size = 32

In [0]:
epochs = 100

In [0]:
number_of_attributes = 40
image_load_shape = (143, 143, 3)
image_shape = (128, 128, 3)

In [0]:
adversarial_loss_mode = 'wgan'

In [0]:
d_gradient_penalty_weight = 10.0
d_attribute_loss_weight = 1.0

In [0]:
g_attribute_loss_weight = 10.0
g_reconstruction_loss_weight = 100.0

In [0]:
load_previous_weights = False
save_current_weights = True

In [0]:
epsilon = 1e-7

# Compute gradient penalty.

In [0]:
def gradient_penalty(discriminator, real_image, fake_image):
    sample_shape = [tf.shape(real_image)[0]] + [1] * (real_image.shape.ndims - 1)
    alpha = tf.random.uniform(shape=sample_shape, minval=0., maxval=1.)

    sample_image = real_image + alpha * (fake_image - real_image)
    sample_image.set_shape(real_image.get_shape().as_list())   
    
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        tape.watch(sample_image)

        predictions = discriminator(sample_image, training=False)
        if isinstance(predictions, tuple):
            predictions = predictions[0]

    gradients = tape.gradient(predictions, sample_image)[0]    
    gradients = tf.reshape(gradients, [tf.shape(gradients)[0], -1])
    norm = tf.norm(epsilon + gradients, axis=1)    
    gp_value = tf.reduce_mean((norm - 1.) ** 2)    

    return(gp_value)

# Load CelebA dataset.

In [0]:
import tensorflow_datasets as tfds

In [0]:
builder = tfds.builder('celeb_a')
print(builder.info)

### Download and prepare dataset.

In [0]:
#builder.download_and_prepare()

# OR

### Download dataset from Google drive.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!cp -r '/content/drive/My Drive/datasets/tensorflow_datasets' /root/.

### View dataset contents.

In [0]:
!ls -al /root/tensorflow_datasets/
!ls -al /root/tensorflow_datasets/celeb_a/

### Load the dataset.

In [0]:
builder.download_and_prepare()

### Create CelebA dataset splits.
* train
* validation
* test

In [0]:
celeba_datasets = builder.as_dataset(shuffle_files=True)
print(celeba_datasets)

In [0]:
train_dataset = celeba_datasets['train']

# Preprocess the dataset.

In [0]:
attributes_to_identifiers = {
    '5_o_Clock_Shadow': 0, 
    'Arched_Eyebrows': 1, 
    'Attractive': 2,       
    'Bags_Under_Eyes': 3,           
    'Bald': 4, 
    'Bangs': 5, 
    'Big_Lips': 6,           
    'Big_Nose': 7, 
    'Black_Hair': 8, 
    'Blond_Hair': 9, 
    'Blurry': 10,           
    'Brown_Hair': 11, 
    'Bushy_Eyebrows': 12, 
    'Chubby': 13,           
    'Double_Chin': 14, 
    'Eyeglasses': 15, 
    'Goatee': 16, 
    'Gray_Hair': 17, 
    'Heavy_Makeup': 18, 
    'High_Cheekbones': 19,          
    'Male': 20, 
    'Mouth_Slightly_Open': 21, 
    'Mustache': 22, 
    'Narrow_Eyes': 23, 
    'No_Beard': 24, 
    'Oval_Face': 25,           
    'Pale_Skin': 26, 
    'Pointy_Nose': 27, 
    'Receding_Hairline': 28,           
    'Rosy_Cheeks': 29, 
    'Sideburns': 30, 
    'Smiling': 31,           
    'Straight_Hair': 32, 
    'Wavy_Hair': 33, 
    'Wearing_Earrings': 34,           
    'Wearing_Hat': 35, 
    'Wearing_Lipstick': 36,           
    'Wearing_Necklace': 37, 
    'Wearing_Necktie': 38, 
    'Young': 39
    }

In [0]:
identifiers_to_attributes = {v: k for k, v in attributes_to_identifiers.items()}

### Create test image.

In [0]:
def create_test_image(image_shape):
  test_image = np.random.rand(image_shape[0], image_shape[1], image_shape[2])
  return(test_image)

### Create test image batch.

In [0]:
def create_test_image_batch(image_shape):
  test_image_batch = np.random.rand(batch_size, image_shape[0], image_shape[1], image_shape[2])
  return(test_image_batch)

### Create test attributes.

In [0]:
def create_test_attributes():
  attributes = np.random.rand(number_of_attributes)
  test_attributes = {}
  for index in range(len(identifiers_to_attributes)):
      attribute = identifiers_to_attributes[index] 
      test_attributes[attribute] = attributes[index]
  return(test_attributes)

### Create test attribute batch.

In [0]:
def create_test_attribute_batch():
  test_attribute_batch = np.random.rand(batch_size, number_of_attributes)
  return(test_attribute_batch)

### Test created test attributes.

In [0]:
test_attributes = create_test_attributes()
print(test_attributes)

### Normalize the image to [-1, 1].

In [0]:
def normalize_image(image):
  image = tf.cast(image, tf.float32)
  image = tf.clip_by_value(image, 0, 255) / 127.5 - 1
  return(image)

### Test image normalization.

In [0]:
input_image = create_test_image(image_shape)
output_image = normalize_image(input_image)
print('input image shape',input_image.shape)
print('output image shape',output_image.shape)

### Random crop the image.

In [0]:
def random_crop(image):
  cropped_image = tf.image.random_crop(image, size=image_shape)
  return(cropped_image)

### Test random croping of image.

In [0]:
input_image = create_test_image(image_load_shape)
output_image = random_crop(input_image)
print('input image shape',input_image.shape)
print('output image shape',output_image.shape)

### Random jitter the image.

In [0]:
def random_jitter(image):  
  image = tf.image.resize(image, [image_load_shape[0], image_load_shape[1]],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)  
  image = random_crop(image)
  image = tf.image.random_flip_left_right(image)
  return(image)

### Test random jittering of image.

In [0]:
input_image = create_test_image(image_load_shape)
output_image = random_jitter(input_image)
print('input image shape',input_image.shape)
print('output image shape',output_image.shape)

### Preprocess train dataset.

In [0]:
def compute_attributes(attributes_batch):
  attributes_array = []
  for index in range(len(identifiers_to_attributes)):
      attribute = identifiers_to_attributes[index]      
      attributes_array.append(tf.cast(attributes_batch[attribute], dtype=tf.float32))      
    
  return(attributes_array)

In [0]:
def preprocess_train_dataset(sample):
  image = sample['image']
  attributes = sample['attributes']
  
  image = random_jitter(image)
  image = normalize_image(image)

  sample['image'] = image
  sample['attributes'] = compute_attributes(attributes)
  return(sample)

### Test the train dataset preprocessing.

In [0]:
input_image = create_test_image(image_load_shape)
attributes = create_test_attributes()

sample = {}
sample['image'] = input_image
sample['attributes'] = attributes

output = preprocess_train_dataset(sample)
print('input image shape',input_image.shape)
print('output image shape',output['image'].shape)

### Preprocess the test dataset.

In [0]:
def preprocess_test_dataset(sample):
  image = sample['image']
  attributes = sample['attributes']

  image = tf.image.resize(image, [image_shape[0], image_shape[1]], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR) 
  image = normalize_image(image)

  sample['image'] = image
  sample['attributes'] = compute_attributes(attributes)
  return(sample)

In [0]:
input_image = create_test_image(image_load_shape)
attributes = create_test_attributes()

sample = {}
sample['image'] = input_image
sample['attributes'] = attributes

output = preprocess_test_dataset(sample)
print('input image shape',input_image.shape)
print('output image shape',output['image'].shape)

### Preprocess dataset splits.

In [0]:
train_dataset = train_dataset.map(preprocess_train_dataset, num_parallel_calls=auto_tune)
train_dataset = train_dataset.shuffle(buffer_size)
train_dataset = train_dataset.batch(batch_size, drop_remainder=True)
train_dataset = train_dataset.prefetch(auto_tune)

# Create the optimizer.

*   Adam optimizer
*   Learning rate = 0.0002
*   β1 = 0.5
*   β2 = 0.999

In [0]:
generator_optimizer = tf.optimizers.Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.999)

discriminator_optimizer = tf.optimizers.Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.999)

# Create the residual block.

In [0]:
import tensorflow_addons as tfa

In [0]:
def create_residual_block(input_layer, residual_dimension):  

    layer = layers.ZeroPadding2D(padding=1)(input_layer)
    layer = layers.Conv2D(filters=residual_dimension, kernel_size=3, strides=1, padding='valid', use_bias=False)(layer)
    layer = tfa.layers.InstanceNormalization(axis=-1)(layer)
    layer = layers.ReLU()(layer)
    layer = layers.ZeroPadding2D(padding=1)(layer) 
    layer = layers.Conv2D(filters=residual_dimension, kernel_size=3, strides=1, padding='valid', use_bias=False)(layer)
    layer = tfa.layers.InstanceNormalization(axis=-1)(layer)
    return( layers.Add()([input_layer, layer]) )

### Test the residual block.

In [0]:
residual_dimension = 64 
shape = (128, 128, 64)
input_layer = layers.Input(shape=shape, name='input_attributes') 
residual_block = create_residual_block(input_layer, residual_dimension)

print(residual_block)

# Create the generator model.

In [0]:
def create_generator_model(image_shape, number_of_attributes, generator_dimension=64, generator_residual_blocks=6): 
        input_image = layers.Input(shape=image_shape, name='input_image')
        input_attributes = layers.Input(shape=number_of_attributes, name='input_attributes')        

        attributes = layers.Lambda(lambda x: K.repeat(x, image_shape[0]*image_shape[1]))(input_attributes)
        attributes = layers.Reshape((image_shape[0], image_shape[0], number_of_attributes))(attributes)
        x = layers.Concatenate()([input_image, attributes])
    
        # First Conv2D
        x = layers.Conv2D(filters=generator_dimension, kernel_size=7, strides=1, padding = 'same', use_bias=False)(x)
        x = tfa.layers.InstanceNormalization(axis=-1)(x)
        x = layers.ReLU()(x)
    
        # Downsampling layers
        current_dimension = generator_dimension
        for i in range(2):
            x = layers.ZeroPadding2D(padding=1)(x)
            x = layers.Conv2D(filters = current_dimension*2, kernel_size=4, strides=2, padding = 'valid', use_bias=False)(x)
            x = tfa.layers.InstanceNormalization(axis=-1)(x)
            x = layers.ReLU()(x)
            current_dimension = current_dimension * 2
        
        # Bottleneck layers.
        for i in range(generator_residual_blocks):
            x = create_residual_block(x, current_dimension)
        
        # Upsampling layers
        for i in range(2):
            x = layers.UpSampling2D(size=2)(x)       
            x = layers.Conv2D(filters = current_dimension//2, kernel_size=4, strides=1, padding='same', use_bias=False)(x)
            x = tfa.layers.InstanceNormalization(axis=-1)(x)
            x = layers.ReLU()(x)        
            current_dimension = current_dimension//2
    
        # Last Conv2D
        x = layers.ZeroPadding2D(padding=3)(x)
        generated_image = layers.Conv2D(filters=3, kernel_size=7, strides=1, padding='valid', activation='tanh', use_bias=False, name='generated_image')(x)
    
        return( models.Model(inputs=[input_image, input_attributes], outputs=generated_image) )

### Test the generator model.

In [0]:
generator = create_generator_model(image_shape, number_of_attributes, generator_dimension, generator_residual_blocks)
generator.summary()

input_image = create_test_image_batch(image_shape)
input_attributes = create_test_attribute_batch()

output_image = generator([input_image, input_attributes])
print('input image shape -', input_image.shape)
print('output image shape -', output_image.shape)

# Create the discriminator / classification model.

In [0]:
def create_discriminator_model(image_shape, number_of_attributes=40, discriminator_dimension=64, dense_dimension=1024, downsamplings_layers=5):
    input_image = layers.Input(shape=image_shape, name='input_image')  
    input_attributes = layers.Input(shape=number_of_attributes, name='input_attributes')  

    output_units = discriminator_dimension
    input_layer = input_image
    
    for layer_index in range(downsamplings_layers): 
        input_layer = layers.Conv2D(output_units, (4,4), strides=(2,2), padding='same')(input_layer)         
        input_layer = tfa.layers.InstanceNormalization()(input_layer)
        input_layer = layers.LeakyReLU(alpha=0.2)(input_layer)

        output_units = output_units * 2

    input_layer = layers.Flatten()(input_layer)

    discriminator_output = layers.Dense(dense_dimension)(input_layer) 
    discriminator_output = tfa.layers.InstanceNormalization()(discriminator_output)     
    discriminator_output = layers.LeakyReLU(alpha=0.2)(discriminator_output)
    discriminator_output = layers.Dense(1, activation=None)(discriminator_output)                  # AttGAN research work - sigmoid_cross_entropy 
    #discriminator_output = layers.Dense(1, activation='sigmoid')(discriminator_output)            # sigmoid - binary_cross_entropy
      
    attribute_output = layers.Dense(dense_dimension)(input_layer)  
    attribute_output = tfa.layers.InstanceNormalization()(attribute_output)         
    attribute_output = layers.LeakyReLU(alpha=0.2)(attribute_output)
    attribute_output = layers.Dense(number_of_attributes, activation=None)(attribute_output)       # AttGAN research work - sigmoid_cross_entropy 
    #attribute_output = layers.Dense(number_of_attributes, activation='sigmoid')(attribute_output) # sigmoid - binary_cross_entropy   

    # Create the discriminator model.
    discriminator_model = models.Model(inputs=[input_image, input_attributes], outputs=[discriminator_output, attribute_output], name='discriminator')

    return(discriminator_model)    

### Test the discriminator model.

In [0]:
input_image = create_test_image_batch(image_shape)

discriminator = create_discriminator_model(image_shape, number_of_attributes=40, discriminator_dimension=64, dense_dimension=1024, downsamplings_layers=5)
discriminator_prediction, attribute_prediction = discriminator(input_image)

print('discriminator prediction shape', discriminator_prediction.shape)
print('attribute prediction shape', attribute_prediction.shape)

# Create adversarial loss functions.
*   Generator loss function
*   Discriminator loss function

## WGAN loss functions.
*   Generator loss function
*   Discriminator loss function

In [0]:
def wgan_loss_functions():
    def discriminator_loss_function(real_logit, fake_logit):
        real_loss = - tf.reduce_mean(real_logit)
        fake_loss = tf.reduce_mean(fake_logit)
        return(real_loss, fake_loss)

    def generator_loss_function(fake_logit):
        fake_loss = - tf.reduce_mean(fake_logit)
        return(fake_loss)

    return(discriminator_loss_function, generator_loss_function)

In [0]:
def adversarial_loss_functions(adversarial_loss_mode):
  if(adversarial_loss_mode == 'wgan'):
    return(wgan_loss_functions())
  else:
    return(wgan_loss_functions())

# Create different models and loss functions.
* Encoder model
* Decoder model
* Discriminator model
* Discriminator loss function
* Generator loss function

In [0]:
encoder = create_encoder_model(image_shape, encoder_dimension=64, downsamplings_layers=5)
decoder = create_decoder_model(encoder, number_of_attributes=40, decoder_dimension=64,)
discriminator = create_discriminator_model(image_shape, number_of_attributes=40, discriminator_dimension=64, dense_dimension=1024, downsamplings_layers=5)

discriminator_loss_function, generator_loss_function = adversarial_loss_functions(adversarial_loss_mode)

# Create the generator model.

In [0]:
input_image = layers.Input(shape=image_shape, name='input_image')  
input_attributes = layers.Input(shape=number_of_attributes, name='input_attributes')

encoded_features = encoder(input_image)
generated_image = decoder([encoded_features, input_attributes])

generator_model = models.Model(inputs=[input_image, input_attributes], outputs=generated_image, name='generator')
generator_model.summary()

### Test the generator model.

In [0]:
input_image = create_test_image_batch(image_shape)
attributes = create_test_attribute_batch()
generated_image = generator_model([input_image, attributes])

print('input image shape',input_image.shape)
print('output image shape',generated_image.shape)

### Load previous model weights.
* Encoder model weights
* Decoder model weights
* Discriminator model weights



In [0]:
!ls -al '/content/drive/My Drive/models/AttGAN/'
!cp '/content/drive/My Drive/models/AttGAN/'*.h5 .
!ls -al

In [0]:
if(load_previous_weights):
  encoder.load_weights('encoder.h5')
  decoder.load_weights('decoder.h5')
  discriminator.load_weights('discriminator.h5')

# Compute the generator loss.

In [0]:
def binary_cross_entropy(y_true, y_pred):
    batch_size = y_true.shape[0]

    loss = - y_true * tf.math.log(y_pred + epsilon) - (1 - y_true) * tf.math.log(1 - y_pred + epsilon)
    loss = tf.reshape(loss, (batch_size, -1))
    loss = tf.reduce_sum(loss, axis=1)
    loss = tf.reduce_mean(loss)

    return loss

In [0]:
def compute_generator_loss(input_image, input_attributes):

  target_attributes = tf.random.shuffle(input_attributes)

  scaled_input_attributes = input_attributes * 2. - 1.
  scaled_target_attributes = target_attributes * 2. - 1.

  # Generator
  input_features = encoder(input_image, training=True)
  reconstructed_image = decoder([input_features, scaled_input_attributes], training=True)
  fake_image = decoder([input_features, scaled_target_attributes], training=True)

  # Discriminator
  fake_image_prediction, fake_image_attributes = discriminator(fake_image, training=False)

  fake_image_prediction_loss = generator_loss_function(fake_image_prediction)

  fake_image_attributes_loss = tf.compat.v1.losses.sigmoid_cross_entropy(target_attributes, fake_image_attributes)   # AttGAN research work - sigmoid_cross_entropy 
  #fake_image_attributes_loss = binary_cross_entropy(target_attributes, fake_image_attributes)                       # sigmoid - binary_cross_entropy
  
  input_image_reconstruction_loss = tf.compat.v1.losses.absolute_difference(input_image, reconstructed_image)
   
  generator_loss = (  fake_image_prediction_loss 
                    + fake_image_attributes_loss * g_attribute_loss_weight 
                    + input_image_reconstruction_loss * g_reconstruction_loss_weight
                    )  

  return(generator_loss)

### Test computation of the generator loss.

In [0]:
sample_batch = next(iter(train_dataset))
loss = compute_generator_loss(sample_batch['image'], sample_batch['attributes'])
print(loss.numpy())

# Compute the discriminator loss.

In [0]:
def compute_discriminator_loss(input_image, input_attributes):

  target_attributes = tf.random.shuffle(input_attributes)

  scaled_input_attributes = input_attributes * 2. - 1.
  scaled_target_attributes = target_attributes * 2. - 1.

  # Generate
  input_features = encoder(input_image, training=False)  
  fake_image = decoder([input_features, scaled_target_attributes], training=False)

  # Discriminate
  input_image_prediction, input_image_attributes = discriminator(input_image, training=True)
  fake_image_prediction, fake_image_attributes = discriminator(fake_image, training=True)

  # Discriminator losses
  input_image_gan_loss, fake_image_gan_loss = discriminator_loss_function(input_image_prediction, fake_image_prediction)  
  gradient_penalty_value = gradient_penalty(discriminator, input_image, fake_image)      

  input_image_attributes_loss = tf.compat.v1.losses.sigmoid_cross_entropy(input_attributes, input_image_attributes) # AttGAN research work - sigmoid_cross_entropy 
  #input_image_attributes_loss = binary_cross_entropy(input_attributes, input_image_attributes)                     # sigmoid - binary_cross_entropy

  discriminator_loss = (  input_image_gan_loss 
                        + fake_image_gan_loss 
                        + gradient_penalty_value * d_gradient_penalty_weight 
                        + input_image_attributes_loss * d_attribute_loss_weight
                        )  
  
  return(discriminator_loss)

### Test computation of the discriminator loss.

In [0]:
sample_batch = next(iter(train_dataset))
loss = compute_discriminator_loss(sample_batch['image'], sample_batch['attributes'])
print(loss.numpy())

# Train the model.

In [0]:
model_loss_frequency = 10
model_save_frequency = 50

In [0]:
def train(train_dataset, epochs=100):  
  generator_loss = None
  discriminator_loss = None

  for epoch in range(epochs):
    batch_index = 0
    
    for dataset_batch in train_dataset:  
      batch_index = batch_index + 1    

      images = dataset_batch['image']   
      attributes = dataset_batch['attributes']
      
      if(batch_index%6 == 0):

        with tf.GradientTape(watch_accessed_variables=False) as encoder_tape, tf.GradientTape(watch_accessed_variables=False) as decoder_tape:
          encoder_tape.watch(encoder.trainable_variables)
          decoder_tape.watch(decoder.trainable_variables)
          generator_loss = compute_generator_loss(images, attributes)

        enoder_gradients = encoder_tape.gradient(generator_loss, encoder.trainable_variables)
        decoder_gradients = decoder_tape.gradient(generator_loss, decoder.trainable_variables)

        encoder_optimizer.apply_gradients(zip(enoder_gradients, encoder.trainable_variables))
        decoder_optimizer.apply_gradients(zip(decoder_gradients, decoder.trainable_variables))

      else:
        with tf.GradientTape(watch_accessed_variables=False) as discriminator_tape:
          discriminator_tape.watch(discriminator.trainable_variables)
          discriminator_loss = compute_discriminator_loss(images, attributes)

        discriminator_gradients = discriminator_tape.gradient(discriminator_loss, discriminator.trainable_variables)
        discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))        

      if(batch_index%model_loss_frequency == 0):
        print('generator loss -', generator_loss.numpy(), 'discriminator loss -', discriminator_loss.numpy())

      if(batch_index%model_save_frequency == 0):
        encoder.save_weights('/content/drive/My Drive/models/AttGAN/encoder.h5')
        decoder.save_weights('/content/drive/My Drive/models/AttGAN/decoder.h5')
        discriminator.save_weights('/content/drive/My Drive/models/AttGAN/discriminator.h5')   
        

In [0]:
train(train_dataset, epochs=10)

In [0]:
if(save_current_weights):
  encoder.save_weights('/content/drive/My Drive/models/AttGAN/encoder.h5')
  decoder.save_weights('/content/drive/My Drive/models/AttGAN/decoder.h5')
  discriminator.save_weights('/content/drive/My Drive/models/AttGAN/discriminator.h5')   